In [15]:
from engine.torchengine import AnalyticalSet, Function, EliminateAnalysis, EliminateAnalysisMergeResiduals, ParallelResiduals, ElimResidual
from engine.torchdata import load_vals, ExpandVector, transfer_value
import torch
import numpy as np
# Set the print options
np.set_printoptions(formatter={'float': lambda x: "{:0.2f}".format(x).rstrip('0').rstrip('.')})

### Orbit calculations (feed forward)

In [2]:
mu, Rearth = 3.986e14, 6378
analysis1 = (('h',), ('a',), lambda h: (h+Rearth)*1e3)
analysis2 = (('a',), ('T',), lambda a: 2*np.pi*(a**3/mu)**0.5)
analysis3 = (('a',), ('g',), lambda a: 1/np.pi*np.arccos(Rearth*1e3/a))
analysis4 = (('g',), ('d',), lambda g: g+0.5)
analysis5 = (('h',), ('r',), lambda h: (h**2+2*Rearth*h)**0.5)

varorder = ['h', 'x', 'R', 'a', 'y', 'T', 'g', 'd', 'r']
indices = {elt: torch.tensor([i]) for i, elt in enumerate(varorder)}

In [3]:
x0 = load_vals({'h': 200, 'a': 1.1*Rearth*1e3, 'g':0.1}, indices, isdict=True)
set1 = AnalyticalSet(analysis1, indices)
set2 = AnalyticalSet(analysis2, indices)
set3 = AnalyticalSet(analysis3, indices)
set4 = AnalyticalSet(analysis4, indices)
set5 = AnalyticalSet(analysis5, indices)
obj = Function((('r',), lambda r: r), indices)

In [4]:
class AnalyticalSetRaw():
    def __init__(self, analysis, residuals, outputs) -> None:
        self.analysis = analysis
        self.residual = residuals
        self.outputs = outputs

In [5]:
orbit_analysis = EliminateAnalysis([set1.analysis, set2.analysis, set3.analysis, set4.analysis, set5.analysis])
orbit_residuals = EliminateAnalysisMergeResiduals(functions=[set1.residual, set2.residual, set3.residual, set4.residual, set5.residual])
all_outputs = [s.analysis.structure[1] for s in [set1, set2, set3, set4, set5]]
Orbit = AnalyticalSetRaw(orbit_analysis, orbit_residuals, all_outputs) # Maybe should be indices too?
Orbit.analysis(x0).numpy()

array([200, 0, 0, 6578000, 0, 5309.48, 0.08, 0.58, 1609.72])

In [6]:
x0.numpy()

array([200, 0, 0, 7015800, 0, 0, 0.1, 0, 0])

In [7]:
inputvar = 'h'
indices_in = {inputvar: indices[inputvar]}
mdf_inputs = ExpandVector(indices_in, indices)
MDF = EliminateAnalysis([mdf_inputs, Orbit.analysis], [obj], flatten=True)
IDF = ParallelResiduals([Orbit.analysis], [obj])

In [8]:
MDF(torch.tensor([200.]))

tensor([1609.7205])

In [9]:
IDF(Orbit.analysis(x0))

[tensor([1609.7205], dtype=torch.float64),
 tensor([0., 0., 0., 0., 0.], dtype=torch.float64)]

In [10]:
IDF(x0)[1].numpy()

array([-437800, 5309.48, -0.02, 0.58, 1609.72])

### Mass calculations

In [18]:
varorder = ['mt', 'ms', 'mp']
indices = {elt: torch.tensor([i]) for i, elt in enumerate(varorder)}

analysis6 = (('mt',), ('ms',), lambda mt: 0.2*mt)
analysis7 = (('ms','mp'), ('mt',), lambda ms, mp: 0.2*ms+mp)
ineqcon1 = (('mp',), lambda mp: -mp/0.5+1)
objfunc = (('mt',), lambda mt: mt)

set6 = AnalyticalSet(analysis6, indices)
set7 = AnalyticalSet(analysis7, indices)
con1 = Function(ineqcon1, indices)
obj = Function(objfunc, indices)

coupled = EliminateAnalysisMergeResiduals(functions=[set6.residual, set7.residual])
solver = ElimResidual(coupled, solvefor=['mt','ms'], indices=indices)
obj_mp = EliminateAnalysis([solver], [obj], flatten=True)

In [22]:
x = torch.tensor([1.,2.,0.7])
obj_mp(x)

tensor([0.7292])

In [23]:
x.numpy()

array([0.73, 0.15, 0.7], dtype=float32)

### Local / global variables

In [11]:
indices1ex = {
    'x': torch.tensor([0, 1, 2]),
    'a1': torch.tensor([3, 4]),
    'a2': torch.tensor([5])
}
indices2ex = {
    'x': torch.tensor([0, 3, 4]),
    'u1': torch.tensor([1, 2]),
    'u2': torch.tensor([5, 6]),
    'a1': torch.tensor([7, 8]),
}
indices2ex_len = len(torch.cat(tuple(indices2ex.values())))
copy_over_indices = ['x']
copy_indices_tuple = [(indices1ex[key], indices2ex[key]) for key in copy_over_indices]

In [12]:
xin = load_vals({'x': torch.tensor([1.,2,3]), 'a1': torch.tensor([4.,5])}, indices1ex, isdict=True)
xzero = torch.empty(indices2ex_len, dtype=xin.dtype)
xout = transfer_value(xin, xzero, copy_indices_tuple)

In [13]:
xout.numpy()

array([1, 0, 0, 2, 3, 0, 0, 0, 0])

In [14]:
set1 = AnalyticalSet((('x',), ('a1',), lambda x: (x[:2],)), indices2ex)
set1.analysis(xout).numpy()

array([1, 0, 0, 2, 3, 0, 0, 1, 2])

### Desired modeling syntax

In [ ]:
Orbit = AnalysisIntersection() # we cannot know what the shared variables will be until all have sets have been added. This is why .analysis and .residual need to be calculated at runtime

h = Var('h', 400e3) 
a = Orbit.Var('a', h + R, setid=1)
T = Orbit.Var('T', 2*np.pi*(a**3/μ)**0.5, setid=2)
g = Orbit.Var('g', 1/np.pi*sp.acos(R/a), setid=3)
d = Orbit.Var('d', g+0.5, setid=4)
r = Orbit.Var('r', (h**2+2*R*h)**0.5, setid=5)

Power = AnalysisIntersection()
eta_A = Par(r'\eta_A', 0.3)
rho_A = Par(r'\rho_A', 10) 
rho_b = Par(r'\rho_b', 0.002)
P_l = Par('P_l', 12, 'W')
A = Var('A', 0.05)
m_A = Power.Var('m_A', rho_A*A, setid=6)
P_c = Power.Var('P_c', d*A*Q*eta_A, setid=7)
P_T = Power.Var('P_T', P_c-P_l, setid=8) 
E_b = Power.Var('E_b', P_c*T/d, setid=9)
m_b = Power.Var('m_b', rho_b*E_b, setid=10)

Payload = AnalysisIntersection()
X_r = Var('X_r', 5)
rho_p = Par(r'\rho_p', 2) 
l_v = Par('l_v', 500, 'nm')
B = Par('B', 8)
N = Par('N', 2000)
D_p = Payload.Var('D_p', 1.22*l_v*h/X_r, setid=11)
D = Payload.Var('D', 2*np.pi*R*B*N/X_r, setid=12)
m_p = Payload.Var('m_p', rho_p*D_p**1.5, setid=13)

# Optimization relevant functions. These are needed for IDF/MDF
ineq = [Xr - 5]
obj = m_p + m_b

# IDF would find shared variables accross all analyses and then generate
# the equality constraints. The resulting form (when using information form one level below) would look like:
([1,2,4,7,9,11,12], 3, 5, 6, 8, 10, 13)